In [ ]:
# %pip install ipympl

In [ ]:
import numpy as np
import caiman as cm
import matplotlib.pyplot as plt
from caiman.motion_correction import MotionCorrect
from caiman.source_extraction.cnmf import cnmf
from caiman.source_extraction.cnmf.params import CNMFParams
from caiman.utils.visualization import view_quilt
import pandas as pd
import sciebo
import bokeh.plotting as bpl
import holoviews as hv
bpl.output_notebook()
hv.notebook_extension('bokeh')
import holoviews as hv

In [ ]:
sciebo.download_file_from_sciebo('https://uni-bonn.sciebo.de/s/RR7qj7tklW1rX25', 'data', 'Sue_2x_3000_40_-46.tif')
sciebo.download_file_from_sciebo('https://uni-bonn.sciebo.de/s/RR7qj7tklW1rX25', 'data', 'data_endoscope.tif')

## Before component evaluation, we need to run motion correction and source extraction

We will do the initial steps using the parameters given directly in the demo notebook of Caiman. 

In [ ]:
# loading movie
fname = "data/Sue_2x_3000_40_-46.tif"
movie_orig = cm.load(fname)

Preparing parameters for component evaluation

In [ ]:
# general dataset-dependent parameters
fr = 30                     # imaging rate in frames per second
decay_time = 0.4            # length of a typical transient in seconds
dxy = (2., 2.)              # spatial resolution in x and y in (um per pixel)

# motion correction parameters
strides = (48, 48)          # start a new patch for pw-rigid motion correction every x pixels
overlaps = (24, 24)         # overlap between patches (width of patch = strides+overlaps)
max_shifts = (6,6)          # maximum allowed rigid shifts (in pixels)
max_deviation_rigid = 3     # maximum shifts deviation allowed for patch with respect to rigid shifts
pw_rigid = True             # flag for performing non-rigid motion correction

# CNMF parameters for source extraction and deconvolution
p = 1                       # order of the autoregressive system (set p=2 if there is visible rise time in data)
gnb = 2                     # number of global background components (set to 1 or 2)
merge_thr = 0.85            # merging threshold, max correlation allowed
bas_nonneg = True           # enforce nonnegativity constraint on calcium traces (technically on baseline)
rf = 15                     # half-size of the patches in pixels (patch width is rf*2 + 1)
stride_cnmf = 10             # amount of overlap between the patches in pixels (overlap is stride_cnmf+1) 
K = 4                       # number of components per patch
gSig = np.array([4, 4])     # expected half-width of neurons in pixels (Gaussian kernel standard deviation)
gSiz = 2*gSig + 1           # Gaussian kernel width and hight
method_init = 'greedy_roi'  # initialization method (if analyzing dendritic data see demo_dendritic.ipynb)
ssub = 1                    # spatial subsampling during initialization 
tsub = 1                    # temporal subsampling during intialization

# parameters for component evaluation
min_SNR = 2.0               # signal to noise ratio for accepting a component
rval_thr = 0.85             # space correlation threshold for accepting a component
cnn_thr = 0.99              # threshold for CNN based classifier
cnn_lowest = 0.1            # neurons with cnn probability lower than this value are rejected

In [ ]:
parameter_dict = {'fnames': fname,
                  'fr': fr,
                  'dxy': dxy,
                  'decay_time': decay_time,
                  'strides': strides,
                  'overlaps': overlaps,
                  'max_shifts': max_shifts,
                  'max_deviation_rigid': max_deviation_rigid,
                  'pw_rigid': pw_rigid,
                  'p': p,
                  'nb': gnb,
                  'rf': rf,
                  'K': K, 
                  'gSig': gSig,
                  'gSiz': gSiz,
                  'stride': stride_cnmf,
                  'method_init': method_init,
                  'rolling_sum': True,
                  'only_init': True,
                  'ssub': ssub,
                  'tsub': tsub,
                  'merge_thr': merge_thr, 
                  'bas_nonneg': bas_nonneg,
                  'min_SNR': min_SNR,
                  'rval_thr': rval_thr,
                  'use_cnn': True,
                  'min_cnn_thr': cnn_thr,
                  'cnn_lowest': cnn_lowest}

parameters = CNMFParams(params_dict=parameter_dict) 

In [ ]:
mc = MotionCorrect(fname, **parameters.motion)
mc.motion_correct(save_movie=True);

In [ ]:
mc_fname = cm.save_memmap(
    mc.fname_tot_els,
    base_name='memmap_',
    order='C'
)

In [ ]:
Yr, dims, num_frames = cm.load_memmap(mc_fname)
images = np.reshape(Yr.T, [num_frames] + list(dims), order='F')
images.shape

## CNMF Model fitting and re-fitting

Below steps can take a while but not as long as before as we have set a lot of parameters: You can listen to this [Music](https://www.youtube.com/watch?v=HImi4zdoZrM)

In [ ]:
cnmf_model = cnmf.CNMF(n_processes=1, params=parameters)

In [ ]:
cnmf_fit = cnmf_model.fit(images)

In [ ]:
cnmf_refit = cnmf_fit.refit(images)

## CNMF Model evaluation


**Signal to Noise Ratio (SNR) `min_SNR`**

What it is: This is a measure of how much the signal (in this case, the activity of neurons as indicated by calcium transients) stands out from the background noise. </br>
How it's done: For each neuron's activity trace (a graph showing their activity over time), a baseline level of noise is determined. Then, the SNR calculates how much the peaks of activity (the calcium traces) stand out compared to this baseline noise. </br>
Why it matters: Higher SNR means the neuron's activity is clearer and more distinct, making it more reliable. High SNR components are considered high quality and are less likely to be mistaken detections (false positives). </br>

**Spatial Correlation `rval_thr`**

What it is: This checks how well the shapes and locations of neurons (their "spatial footprints") detected in the data match up with where and when actual neuron activity is seen in the video (movie) of the brain. </br>
How it's done: The spatial footprints extracted are compared to the actual neuron activity in the video. This comparison generates correlation coefficients (values that measure how similar two patterns are) for the times when the neurons are active. </br>
Why it matters: High correlation means the detected spatial footprints accurately represent real neuron activity, which is crucial for valid analysis. </br>


**CNN Confidence `min_cnn_thr`**

What it is: This uses a Convolutional Neural Network (CNN) to evaluate whether the detected shapes of neurons are likely to be real. </br>
How it's done: Each detected spatial component (neuron shape) is analyzed by the CNN, which has been trained on a large set of data where the correct answers (which shapes are truly neurons) are already known. </br>
Why it matters: The CNN gives a confidence score between 0 and 1 for each shape. Scores closer to 1 indicate the shape is very likely to be a real neuron. </br>



These parameters can be set in the `quality` field of `params`. Let's see that they are set to the values we assigned before. 

What is the min_SNR, rval_thr, min_cnn_thr used for evaluation?

In [ ]:
cnmf_refit.params.quality['min_SNR']

In [ ]:
cnmf_refit.params.quality['rval_thr']

In [ ]:
cnmf_refit.params.quality['min_cnn_thr']

The evaluate_components() is the method that uses the min_SNR, rval_thr, and min_cnn_thr to `accept` or `reject` the identified components. If an identified component is below all of the three values, it will be rejected.

In [ ]:
cnmf_refit.estimates.evaluate_components(images, cnmf_refit.params);

The `evaluate_components` method creates `idx_components` (accepted) and `idx_components_bad` (rejected) fields in the `Estimates` class.

Let's see how many accepted and rejected components

In [ ]:
len(cnmf_refit.estimates.idx_components)

In [ ]:
len(cnmf_refit.estimates.idx_components_bad)

Let's plot the contours of both accepted and rejected components

In [ ]:
correlation_image = cm.local_correlations(images, swap_dim=False)
cnmf_refit.estimates.plot_contours_nb(img=correlation_image, idx=cnmf_refit.estimates.idx_components)

There are also some visualizations that you can use to visualize denoised calcium traces of each component and their SNR

Accepted components

In [ ]:
correlation_image = cm.local_correlations(images, swap_dim=False)
cnmf_refit.estimates.nb_view_components(img=correlation_image, idx=cnmf_refit.estimates.idx_components, cmap='gray', denoised_color='red');

Bad components

In [ ]:
correlation_image = cm.local_correlations(images, swap_dim=False)
cnmf_refit.estimates.nb_view_components(img=correlation_image, idx=cnmf_refit.estimates.idx_components_bad, cmap='gray', denoised_color='red')

1. Component evaluation of 1-photon data. The procedure is similar to the one we followed in the notebook. If you want some data for setting parameters, here is a link to the caiman tutorial on [CNMF-E Demo](https://github.com/flatironinstitute/CaImAn/blob/main/demos/notebooks/demo_pipeline_cnmfE.ipynb)

2. Exercise: delta F/F

Follow `Extract delta F/F` section of [CNMF Demo](https://github.com/flatironinstitute/CaImAn/blob/main/demos/notebooks/demo_pipeline.ipynb)
